In [1]:
from env_vars import *

In [2]:
from enum import Enum
from pprint import pprint

import dspy
from pydantic import BaseModel, Field
from lingua import LanguageDetectorBuilder

from tools.search_hadith import SearchHadith
from tools.search_quran import SearchQuran
from tools.search_mawsuah import SearchMawsuah

In [3]:
import dspy
import openai
import backoff

class OpenAIWrapper(dspy.dsp.modules.gpt3.GPT3):
    @backoff.on_exception(
        backoff.expo,
        (openai.RateLimitError, openai.InternalServerError),
        max_time=1000,
        on_backoff=dspy.dsp.modules.gpt3.backoff_hdlr,
    )
    def request(self, prompt: str, **kwargs):
        """Handles retreival of GPT-3 completions whilst handling rate limiting and caching."""
        if "model_type" in kwargs:
            del kwargs["model_type"]

        return self.basic_request(prompt, **kwargs)

In [4]:
# Stage 1
class SearchSource(str, Enum):
    HADITH = "Hadith"
    QURAN = "Quran"
    MAWSUAH = "Mawsuah"
    NONE = "None"

class Stage1InputQuery(BaseModel):
    query: str = Field(description="Islam-related question")

class Stage1OutputResult(BaseModel):
    language: str = Field(description="Language of the question")
    search_queries: dict[SearchSource, str] = Field(description="Dictionary containing search queries for each source in Arabic")


class PrecisionIslamQueries(dspy.Signature):
    """Continually enhance your language capabilities for the Holy Quran, Hadith, and Mawsuah, an encyclopedia of Islamic jurisprudence (fiqh), by practicing effective search queries. Engage consistently with diverse questions posing challenges, analyzing them thoroughly to craft authentic Arabic search queries. Regular interaction with different topics ensures a richer vocabulary and precise queries targeting each source appropriately."""
    input: Stage1InputQuery = dspy.InputField()
    output: Stage1OutputResult = dspy.OutputField()


In [5]:
# Stage 2
class Stage2InputData(BaseModel):
    question: str = Field(description="An Islam-related question")
    language: str = Field(description="Language of the question")
    search_results: dict[SearchSource, list[str]] = Field(description="Dictionary containing search results for each source")

class Stage2OutputAnswer(BaseModel):
    answer: str = Field(description="Final answer to the question")

class GenerateAnswer(dspy.Signature):
    """
    You are Ansari, a multilingual Islamic bot. Answer questions with thorough, well-researched answers, grounded in the rich tradition of Sunni scholarship.
    """

    input: Stage2InputData = dspy.InputField()
    output: Stage2OutputAnswer = dspy.OutputField()

In [6]:
class AnsariFlow(dspy.Module):
    def __init__(self, openai_model_name="gpt-4o"):
        self.llm = OpenAIWrapper(model=openai_model_name, api_key=OPENAI_API_KEY, max_tokens=2000, model_type='chat')
        self.ms = SearchMawsuah(auth_token=VECTARA_AUTH_TOKEN, customer_id=VECTARA_CUSTOMER_ID, corpus_id=VECTARA_CORPUS_ID)
        self.hs = SearchHadith(api_key=KALEMAT_API_KEY)
        self.qs = SearchQuran(api_key=KALEMAT_API_KEY)
        self.stage1_predictor = dspy.TypedPredictor(PrecisionIslamQueries, max_retries=5, explain_errors=True)
        self.stage2_predictor = dspy.TypedPredictor(GenerateAnswer)
        

    def forward(self, question: str):
        # Stage 1: Generate search query or provide a direct answer if possible
        with dspy.context(lm=self.llm):
            stage1_input = Stage1InputQuery(query=question)
            stage1_prediction = self.stage1_predictor(input=stage1_input)
            # pprint(stage1_prediction.output)

        # Stage 1 search queries and results
        search_results = {source: [] for source in SearchSource}

        for source, search_query in stage1_prediction.output.search_queries.items():
            if source == SearchSource.HADITH:
                search_results[source] = self.hs.run_as_list(search_query)
            elif source == SearchSource.QURAN:
                search_results[source] = self.qs.run_as_list(search_query)
            elif source == SearchSource.MAWSUAH:
                search_results[source] = self.ms.run_as_list(search_query)

        # Stage 2: Generate the final answer using search results
        with dspy.context(lm=self.llm):
            stage2_input = Stage2InputData(question=question, language=stage1_prediction.output.language, search_results=search_results)
            stage2_prediction = self.stage2_predictor(input=stage2_input)
            # pprint(stage2_prediction.output)
        return dspy.Prediction(answer=stage2_prediction.output.answer, quran_search_results="||".join(search_results[SearchSource.QURAN]), hadith_search_results="||".join(search_results[SearchSource.HADITH]), mawsuah_search_results="||".join(search_results[SearchSource.MAWSUAH]), target_language=stage1_prediction.output.language)
    

In [7]:
# question = "kadinlarin basortusu takmalari sart mi?"
# ansari = AnsariFlow(openai_model_name="gpt-3.5-turbo")
# answer = ansari(question)
# pprint(answer)

In [8]:
# question = "kadinlarin basortusu takmalari sart mi?"
# ansari4o = AnsariFlow(openai_model_name="gpt-4o")
# answer = ansari4o(question)
# pprint(answer)

In [9]:
# answer = ansari4o("Who is Allah?")
# pprint(answer.answer)

In [10]:
class RelevanceSignature(dspy.Signature):
    """Assess the relevance of an answer."""

    question: str = dspy.InputField(desc="The question asked.")
    predicted_answer: str = dspy.InputField(desc="The answer given by the system.")

    relevance: str = dspy.OutputField(desc="How relevant is the answer to the question? Answer with one of: high, moderate, or low.")

class SemanticAndMentionsSignature(dspy.Signature):
    """Assess the semantic match and mentions of Ayah and Hadith in an answer."""

    ground_truth_answer: str = dspy.InputField(desc="The expert's correct answer.")
    predicted_answer: str = dspy.InputField(desc="The answer given by the system.")

    semantic_match: str = dspy.OutputField(desc="Do the system answer and the ground truth answer convey the same meaning? Answer with 'Yes' or 'No'.")
    mentions_ayah: str = dspy.OutputField(desc="Does the answer mention an Ayah? Answer with 'Yes' or 'No'.")
    mentions_hadith: str = dspy.OutputField(desc="Does the answer mention a Hadith? Answer with 'Yes' or 'No'.")

class FluencyAndAddressSignature(dspy.Signature):
    """Assess the fluency and completeness of an answer."""

    predicted_answer: str = dspy.InputField(desc="The answer given by the system.")

    is_concise: str = dspy.OutputField(desc="Is the predicted answer concise, succinct and to the point? Answer with 'Yes' or 'No'.")
    eloquent: str = dspy.OutputField(desc="Is the answer clear, fluent, engaging, native-sounding, articulate, smooth, easy to read, and easy to understand? Answer with 'Yes' or 'No'.")
    fully_addresses_question: str = dspy.OutputField(desc="Does the answer fully address the question? Answer with 'Yes' or 'No'.")

class TraditionAndToneSignature(dspy.Signature):
    """Assess the tradition and tone of an answer."""

    predicted_answer: str = dspy.InputField(desc="The answer given by the system.")

    sunni_tradition: str = dspy.OutputField(desc="Is the answer based on Sunni tradition? Answer with 'Yes' or 'No'.")
    gentle_tone: str = dspy.OutputField(desc="Is the tone gentle and non-judgmental? Answer with 'Yes' or 'No'.")

class QuranFormatSignature(dspy.Signature):
    """Evaluate the evidence citation and formatting of an answer."""

    predicted_answer: str = dspy.InputField(desc="The answer given by the system.")
    quran_search_results: list[str] = dspy.InputField(desc="List of search results from the Quran.")
    target_language: str = dspy.InputField(desc="The target language for the answer.")

    quran_format: str = dspy.OutputField(desc="Is the Quranic reference correctly formatted, including the ayah number, Arabic text, and translation (if the target language differs from Arabic)? Answer with 'Yes' or 'No'.")
    quran_quoting: str = dspy.OutputField(desc="Are the referenced ayahs quoted exclusively from the search results? Answer with 'Yes' or 'No'.")
    quran_translated: str = dspy.OutputField(desc="Are the mentioned ayahs translated into the target language? Answer with 'Yes' or 'No'.")

class HadithFormatSignature(dspy.Signature):
    """Evaluate the evidence citation and formatting of an answer."""

    predicted_answer: str = dspy.InputField(desc="The answer given by the system.")
    hadith_search_results: list[str] = dspy.InputField(desc="List of search results from the Hadith.")
    target_language: str = dspy.InputField(desc="The target language for the answer.")

    hadith_quoting: str = dspy.OutputField(desc="Are the referenced hadiths quoted exclusively from the search results? Answer with 'Yes' or 'No'.")
    hadith_format: str = dspy.OutputField(desc="Is the Hadith reference correctly formatted, including the collection, LK id, text, and grade? Answer with 'Yes' or 'No'.")
    external_hadith_format: str = dspy.OutputField(desc="Is the format for Hadiths not from the search results correct, as follows: 'I believe (though not 100% sure of the reference) there is a hadith that says: [text of hadith]'? Answer with 'Yes' or 'No'.")

class EvidenceCitationSignature(dspy.Signature):
    """Evaluate the evidence citation and formatting of an answer."""

    predicted_answer: str = dspy.InputField(desc="The answer given by the system.")
    cites_evidence: str = dspy.OutputField(desc="Is each assertion made in the answer directly supported by evidence? Answer with 'Yes' or 'No'.")


class ScholarReferenceSignature(dspy.Signature):
    """Assess the scholar references in an answer."""

    predicted_answer: str = dspy.InputField(desc="The answer given by the system.")

    vague_scholar_reference: str = dspy.OutputField(desc="Does the answer use vague scholar references? Answer with 'Yes' or 'No'.")
    cites_classical_scholars: str = dspy.OutputField(desc="Does the answer cite classical scholars? Answer with 'Yes' or 'No'.")
    cites_modern_scholars: str = dspy.OutputField(desc="Does the answer cite modern scholars? Answer with 'Yes' or 'No'.")



In [11]:
llm_evaluator = OpenAIWrapper(model='gpt-3.5-turbo', max_tokens=2000, model_type='chat', api_key=OPENAI_API_KEY)
llm_gpt4o = OpenAIWrapper(model='gpt-4o', max_tokens=2000, model_type='chat', api_key=OPENAI_API_KEY)

In [12]:
def evaluate_relevance(question, predicted_answer):
    with dspy.context(lm=llm_evaluator):
        evaluation = dspy.Predict(RelevanceSignature)(
            question=question,
            predicted_answer=predicted_answer
        )
        if 'high' in evaluation.relevance.lower():
            return 2
        elif 'moderate' in evaluation.relevance.lower():
            return 1
        elif 'low' in evaluation.relevance.lower():
            return -1
        else:
            return 0

def evaluate_semantic_and_mentions(ground_truth_answer, predicted_answer):
    with dspy.context(lm=llm_evaluator):
        evaluation = dspy.Predict(SemanticAndMentionsSignature)(
            ground_truth_answer=ground_truth_answer,
            predicted_answer=predicted_answer
        )
    return ["yes" in eval.lower() for eval in [evaluation.semantic_match, evaluation.mentions_ayah, evaluation.mentions_hadith]]

def evaluate_fluency_and_address(predicted_answer):
    with dspy.context(lm=llm_evaluator):
        evaluation = dspy.Predict(FluencyAndAddressSignature)(
            predicted_answer=predicted_answer
        )
    return ["yes" in eval.lower() for eval in [evaluation.is_concise, evaluation.eloquent, evaluation.fully_addresses_question]]

def evaluate_tradition_and_tone(predicted_answer):
    with dspy.context(lm=llm_evaluator):
        evaluation = dspy.Predict(TraditionAndToneSignature)(
            predicted_answer=predicted_answer
        )
    return ["yes" in eval.lower() for eval in [evaluation.sunni_tradition, evaluation.gentle_tone]]

def evaluate_quran_format(predicted_answer, quran_search_results, target_language):
    with dspy.context(lm=llm_evaluator):
        evaluation = dspy.Predict(QuranFormatSignature)(
            predicted_answer=predicted_answer,
            quran_search_results=quran_search_results,
            target_language=target_language
        )
    return ["yes" in eval.lower() for eval in [evaluation.quran_format, evaluation.quran_quoting, evaluation.quran_translated]]

def evaluate_hadith_format(predicted_answer, hadith_search_results, target_language):
    with dspy.context(lm=llm_evaluator):
        evaluation = dspy.Predict(HadithFormatSignature)(
            predicted_answer=predicted_answer,
            hadith_search_results=hadith_search_results,
            target_language=target_language
        )
    return ["yes" in eval.lower() for eval in [evaluation.hadith_quoting, evaluation.hadith_format, evaluation.external_hadith_format]]

def evaluate_evidence(predicted_answer):
    with dspy.context(lm=llm_evaluator):
        evaluation = dspy.Predict(EvidenceCitationSignature)(
            predicted_answer=predicted_answer
        )
    return ["yes" in eval.lower() for eval in [evaluation.cites_evidence]]

def evaluate_scholar_reference(predicted_answer):
    with dspy.context(lm=llm_evaluator):
        evaluation = dspy.Predict(ScholarReferenceSignature)(
            predicted_answer=predicted_answer
        )
    return ["yes" in eval.lower() for eval in [evaluation.cites_classical_scholars, evaluation.cites_modern_scholars]]


def calculate_score(evaluations):
    return sum(1 if evaluation else -1 for evaluation in evaluations)


In [13]:
lang_detector = LanguageDetectorBuilder.from_all_languages().with_preloaded_language_models().build()

def metric(gold, pred, trace=None):
    question, ground_truth_answer = gold.question, gold.answer
    evaluations = []
    evaluations.extend(evaluate_semantic_and_mentions(ground_truth_answer, pred.answer))
    evaluations.extend(evaluate_fluency_and_address(pred.answer))
    evaluations.extend(evaluate_tradition_and_tone(pred.answer))
    evaluations.extend(evaluate_quran_format(pred.answer, pred.quran_search_results, pred.target_language))
    evaluations.extend(evaluate_hadith_format(pred.answer, pred.hadith_search_results, pred.target_language))
    evaluations.extend(evaluate_evidence(pred.answer))
    evaluations.extend(evaluate_scholar_reference(pred.answer))
    evaluations.append(lang_detector.detect_language_of(question)==lang_detector.detect_language_of(pred.answer))
    score = calculate_score(evaluations)
    score += evaluate_relevance(question, pred.answer)
    return score

In [14]:
import json
import random
with open("yaqeen-what-islam-says-about.json", "r") as f:
    qa_pairs = json.load(f)["questions"]
    random.seed(2024)
    random.shuffle(qa_pairs)
    
data = []
for pair in qa_pairs:
    question, answer = pair["question"], pair["answer"]
    data.append(dspy.Example(question=question, answer=answer).with_inputs("question"))

train_set_size = int(len(qa_pairs) * 0.9)
trainset, testset = data[:train_set_size], data[train_set_size:]


In [15]:
# # if want to evaluate on devset before optimizing
# from dspy.evaluate import Evaluate

# NUM_THREADS = 5
# evaluate = Evaluate(devset=devset, metric=metric, num_threads=NUM_THREADS, display_progress=True, display_table=False)
# evaluate(AnsariFlow(openai_model_name="gpt-3.5-turbo"), devset=trainset)

In [16]:
dspy.settings.configure(lm=llm_evaluator)

In [17]:
from dspy.teleprompt import COPRO

teleprompter = COPRO(
    metric=metric,
    verbose=True,
)
##
kwargs = dict(num_threads=8, display_progress=True, display_table=0) # Used in Evaluate class in the optimization process
compiled_prompt_opt = teleprompter.compile(AnsariFlow(openai_model_name="gpt-4o"), trainset=trainset, eval_kwargs=kwargs)
##
compiled_prompt_opt.save("optimized_prompts_05_4o_minimal_init_prompt.json")

2024-06-04T12:25:27.103485Z [warning  ] DeprecationWarning: 'verbose' has been deprecated. To see all information for debugging, use 'dspy.set_log_level('debug')'. In the future this will raise an error. [dspy.teleprompt.copro_optimizer] filename=copro_optimizer.py lineno=78


Searching quran for "هل يسمح الإسلام بجرائم الشرف؟"
Searching quran for "هل يقدم القرآن توجيهات حول كيفية التعامل مع الصدمة والألم؟"
Searching quran for "أين الكعبة؟"


  0%|          | 0/98 [00:00<?, ?it/s]

Searching quran for "من يجب عليه أداء الحج؟"Searching quran for "هل يصوم المسلمون خارج رمضان؟"

Searching quran for "ما هي الفتوى؟"
Searching quran for "هل الردة يعاقب عليها بالإعدام في الإسلام؟"
Searching quran for "ما معنى حلال؟"
Searching hadith for "أين الكعبة؟"
Searching hadith for "ما معنى حلال؟"
Searching hadith for "هل توجد أحاديث تتعلق بجرائم الشرف؟"
Searching hadith for "من يجب عليه أداء الحج؟"
Searching hadith for "هل يعاقب المرتد بالإعدام في الإسلام؟"
Searching hadith for "ما هي الفتوى؟"
Searching hadith for "هل يصوم المسلمون خارج رمضان؟"
Searching hadith for "هل تقدم السنة النبوية توجيهات حول كيفية التعامل مع الصدمة والألم؟"
Searching al-mawsuah for "ما هو حكم الفقه الإسلامي في جرائم الشرف؟"
Searching al-mawsuah for "من يجب عليه أداء الحج؟"
Searching al-mawsuah for "أين الكعبة؟"
Searching al-mawsuah for "ما معنى حلال؟"
Searching al-mawsuah for "ما هي الفتوى؟"
Searching al-mawsuah for "حكم الردة في الفقه الإسلامي"
Searching al-mawsuah for "هل يصوم المسلمون خارج رمضان؟"
Sear

Average Metric: 14 / 2  (700.0):   2%|▏         | 2/98 [00:13<09:06,  5.69s/it]

...

Average Metric: 496.0 / 64  (775.0):  65%|██████▌   | 64/98 [00:45<01:51,  3.29s/it]

Searching quran for "ما معنى الإسلام؟"
Searching hadith for "ما معنى الإسلام في الحديث؟"


ERROR:dspy.evaluate.evaluate:2024-06-04T12:44:05.637029Z [error    ] Error for example in dev set: 		 HTTPSConnectionPool(host='api.vectara.io', port=443): Max retries exceeded with url: /v1/query (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ed1a8513100>: Failed to establish a new connection: [Errno -2] Name or service not known')) [dspy.evaluate.evaluate] filename=evaluate.py lineno=147
Average Metric: 496.0 / 67  (740.3):  67%|██████▋   | 66/98 [00:46<01:36,  3.03s/it]

Searching quran for "ما هو عيد الفطر؟"
Searching quran for "لماذا الحج مهم؟"
Searching quran for "كيفية أداء الحج"
Searching quran for "هل يقول القرآن بقتل غير المؤمنين؟"
Searching al-mawsuah for "ما هو تعريف الإسلام في الفقه الإسلامي؟"
Searching quran for "هل هناك استثناءات أو إعفاءات من الصيام؟"
Searching quran for "ما هي العمرة؟"
Searching quran for "هل انتشر الإسلام بالعنف والإكراه؟"
Searching hadith for "كيفية أداء الحج"
Searching hadith for "ما هو عيد الفطر؟"
Searching hadith for "هل يوجد حديث يأمر بقتل غير المؤمنين؟"
Searching hadith for "أهمية الحج في الأحاديث"
Searching hadith for "ما هي العمرة؟"
Searching hadith for "هل هناك استثناءات أو إعفاءات من الصيام؟"
Searching hadith for "هل انتشر الإسلام بالعنف والإكراه؟"
Searching quran for "هل العمرة والحج نفس الشيء؟"
Searching al-mawsuah for "كيفية أداء الحج"
Searching al-mawsuah for "ما هو حكم قتل غير المؤمنين في الفقه الإسلامي؟"
Searching al-mawsuah for "ما هو عيد الفطر؟"
Searching al-mawsuah for "أهمية الحج في الفقه الإسلامي"
Se

ConnectionError: HTTPSConnectionPool(host='api.vectara.io', port=443): Max retries exceeded with url: /v1/query (Caused by NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x7ed1a85119f0>: Failed to establish a new connection: [Errno -2] Name or service not known'))

In [ ]:
answer = """
In Islam, men and women are considered equal in terms of their spiritual worth and responsibilities before Allah. However, they have different roles and responsibilities in certain aspects of life. This concept of equality with distinction is rooted in the Qur'an and Hadith.

### Equality in Spiritual Worth

The Qur'an emphasizes that both men and women are equally capable of attaining spiritual success and are equally accountable for their actions:

Ayah: 33:35
Arabic: إِنَّ الْمُسْلِمِينَ وَالْمُسْلِمَاتِ وَالْمُؤْمِنِينَ وَالْمُؤْمِنَاتِ وَالْقَانِتِينَ وَالْقَانِتَاتِ وَالصَّادِقِينَ وَالصَّادِقَاتِ وَالصَّابِرِينَ وَالصَّابِرَاتِ وَالْخَاشِعِينَ وَالْخَاشِعَاتِ وَالْمُتَصَدِّقِينَ وَالْمُتَصَدِّقَاتِ وَالصَّائِمِينَ وَالصَّائِمَاتِ وَالْحَافِظِينَ فُرُوجَهُمْ وَالْحَافِظَاتِ وَالذَّاكِرِينَ اللَّهَ كَثِيرًا وَالذَّاكِرَاتِ أَعَدَّ اللَّهُ لَهُم مَّغْفِرَةً وَأَجْرًا عَظِيمًا
Translation: "Indeed, the Muslim men and Muslim women, the believing men and believing women, the obedient men and obedient women, the truthful men and truthful women, the patient men and patient women, the humble men and humble women, the charitable men and charitable women, the fasting men and fasting women, the men who guard their private parts and the women who do so, and the men who remember Allah often and the women who do so - for them Allah has prepared forgiveness and a great reward."

### Different Roles and Responsibilities

While men and women are equal in their spiritual worth, they have different roles and responsibilities in certain aspects of life. For example, men are generally given the role of financial providers, while women are often given the role of caregivers. This distinction is not meant to imply superiority or inferiority but rather to ensure a balanced and functional society.

Ayah: 4:34
Arabic: الرِّجَالُ قَوَّامُونَ عَلَى النِّسَاءِ بِمَا فَضَّلَ اللَّهُ بَعْضَهُمْ عَلَىٰ بَعْضٍ وَبِمَا أَنفَقُوا مِنْ أَمْوَالِهِمْ
Translation: "Men are the protectors and maintainers of women, because Allah has given the one more (strength) than the other, and because they support them from their means."

### Scholarly Opinions

Classical scholars like Imam Al-Ghazali and Ibn Taymiyah have emphasized that the different roles assigned to men and women are complementary and are designed to create a balanced and just society. Modern scholars like Yasir Qadhi and Hamza Yusuf also stress that these roles should not be seen as a form of discrimination but rather as a means to ensure that both men and women can fulfill their potential in different aspects of life.

### Conclusion

In summary, Islam teaches that men and women are equal in their spiritual worth and accountability before Allah, but they have different roles and responsibilities in certain aspects of life. This distinction is meant to create a balanced and functional society, not to imply any form of superiority or inferiority.
"""
question = "Are men and women equal in Islam?"
yaqeen_answer = """Every man and woman is equal in their humanity and dignity before God.

In Islam, nobility and distinction are a function of piety, not of gender, race, wealth, or any other worldly trait.

O people, We created you from a male and a female, and made you into nations and tribes so that you may get to know one another. Surely the most noble of you in the sight of God is the most righteous among you. God is truly All-Knowing, All-Aware.

[Quran, 49:13]

The Prophet Muhammad ﷺ also declared that, “Women are the shaqā’iq (full siblings) of men.”"""